In [5]:
import pandas as pd
import requests
import json
import datetime
import csv
from pmaw import PushshiftAPI

In [ ]:
# One thing to note, I get more features/columns when I use the url as opposed to the 
# PushshiftAPI()

In [99]:
api = PushshiftAPI()
comments = api.search_comments(q='python', limit=25)
comments_df = pd.DataFrame(comments)
len(comments_df.columns)

Not all PushShift shards are active. Query results may be incomplete.


36

In [97]:
# This is for comment data
query = 'python'
def get_pushshift_comment_data(query):
    url = 'https://api.pushshift.io/reddit/comment/search/?q='+query
    print(url)
    r = requests.get(url)
    data = json.loads(r.text, strict=False)
    return data['data']

len(get_pushshift_comment_data(query)[0].keys())
# get_pushshift_comment_data(query)[0].keys()

https://api.pushshift.io/reddit/comment/search/?q=python


47

In [98]:
# This is for submission data
query = 'python'
def get_pushshift_submission_data(query):
    url = 'https://api.pushshift.io/reddit/search/submission/?q='+query
    print(url)
    r = requests.get(url)
    data = json.loads(r.text, strict=False)
    return data['data']

len(get_pushshift_submission_data(query)[0].keys())
# get_pushshift_submission_data(query)[0].keys()

https://api.pushshift.io/reddit/search/submission/?q=python


57

In [90]:
# I didn't include all features here but we certianly could or pick and choose
def collect_com_data(coms):
    com_data = list()
    author = coms['author']
    author_name = coms['author_fullname']
    try:
        flair = coms['author_flair_text']
    except KeyError:
        flair = 'NaN'
    try:
        body = coms['body']
    except KeyError:
        body = ''
    com_id = coms['id']
    score = coms['score']
    created_at = datetime.datetime.fromtimestamp(coms['created_utc'])
    try:
        controversiality = coms['controversiality']
    except KeyError:
        controversiality = 'NaN'
    subreddit = coms['subreddit']
    
    com_data.append((com_id,author,author_name,body,subreddit,score,controversiality,created_at,flair))
    com_stats[com_id] = com_data

# This will write the collected data to a CSV
def update_com_file():
    upload_count = 0
    print('input filename, please add .csv')
    filename = input()
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file:
        a = csv.writer(file, delimiter=',')
        headers = ['ID','Author','Name','Text','Subreddit','Score','Controversial','Publish Date','Flair']
        a.writerow(headers)
        for com in com_stats:
            a.writerow(com_stats[com][0])
            upload_count += 1
            
        print(str(upload_count) + ' comments have been uploaded into a csv file')
     

In [91]:
com_stats = {}
com_count = 0
query = 'science'
# Besides query we can also use other filter metrics like: before, after, limit, etc.

data = get_pushshift_data(query)

# I still need to work on this while loop but for the moment it works for testing
while len(data) > 0:
    for comment in data:
        collect_com_data(comment)
        com_count+=1
    print(len(data))

    data = get_pushshift_data(query)
    break

print(len(data))

update_com_file()
# For some reason the "Text" or "body" of the comment is not being collected and written 
# to the CSV. It does work when I search submission instead of comment. I'm in the process
# of figuring out why.


https://api.pushshift.io/reddit/search/submission/?q=science
25
https://api.pushshift.io/reddit/search/submission/?q=science
25
input filename, please add .csv
t.csv
25 comments have been uploaded into a csv file
